Base imports.

In [1]:
import os
import json
import pandas as pd
from pprint import pprint

from helpers import _set_env
_set_env("OPENAI_API_KEY")
_set_env("ANTHROPIC_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("LINKEDIN_COOKIE_LI_AT")
_set_env("LINKEDIN_COOKIE_JSESSIONID")

import warnings
warnings.filterwarnings('ignore')

agent_name = "podcast_outline"
collected_data_dir = f"collected_data/{agent_name}"
os.makedirs(collected_data_dir, exist_ok=True)
base_data_dir = f"base_data/{agent_name}"
os.makedirs(base_data_dir, exist_ok=True)

Import CrewAI, tools, and LLM / AI agent-related dependencies.

In [2]:
from crewai import Agent, Task, Crew, LLM
from crewai.tools import BaseTool
from crewai_tools import FileReadTool, ScrapeWebsiteTool, YoutubeVideoSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from pydantic import BaseModel, Field
from typing import Optional

Initialize the LLM to be used. 

*NOTE: Later you can customize or use different LLMs for each agent or task.*

In [3]:
llm = LLM(model="anthropic/claude-3-5-sonnet-20241022", temperature=1.0)

Connect to LinkedIn API.

In [4]:
with open("linkedin_credentials.json", "r") as f:
    credentials = json.load(f)

from requests.cookies import cookiejar_from_dict
cookies = cookiejar_from_dict({
    "liap": "true",
    "li_at": os.environ["LINKEDIN_COOKIE_LI_AT"],
    "JSESSIONID": os.environ["LINKEDIN_COOKIE_JSESSIONID"],
})

from linkedin_api import Linkedin
linkedin = Linkedin(credentials["username"], credentials["password"], cookies=cookies)

Instantiate tools.

In [7]:
file_read_tool = FileReadTool()
scrape_website_tool = ScrapeWebsiteTool()
youtube_video_search_tool = YoutubeVideoSearchTool()

Create custom tool for Tavily search -- this is supposed to work better than Serper API and other search tools as it's LLM-tailored.

In [8]:
class WebSearchTool(BaseTool):
    name: str ="Web Search Tool"
    description: str = "Search the web for current information on a given topic or person"
        
    def _run(self, query: str, max_results: Optional[int] = 10) -> str:
        # Perform the search
        search_client = TavilySearchResults(max_results=max_results)
        search_results = search_client.invoke(query)
        
        # Format the results
        formatted_results = []
        for doc in search_results:
            formatted_doc = f"URL: {doc['url']}\n"
            formatted_doc += f"Content: {doc['content']}\n"
            formatted_results.append(formatted_doc)
            
        return "\n---\n".join(formatted_results)

In [9]:
web_search_tool = WebSearchTool()

Create agents.

In [10]:
# Create the researcher agent
researcher = Agent(
    role='Senior Podcast Guest Researcher',
    goal='Thoroughly research the guest to understand their expertise, background, and recent work',
    backstory="""You are an expert podcast researcher who excels at finding and synthesizing 
    information about podcast guests. You analyze their background, work, and online presence
    to understand their unique perspectives and contributions to the specific topic of the podcast.""",
    tools=[file_read_tool],
    llm=llm,
    allow_delegation=False,
    verbose=True,
)

# Create the outline writer agent
outline_writer = Agent(
    role='Senior Podcast Outline Writer',
    goal='Create engaging, personalized podcast outlines that highlight the guest\'s expertise',
    backstory="""You are an experienced podcast producer who knows how to structure 
    compelling conversations. You excel at creating outlines that flow naturally and 
    bring out the best in each guest.""",
    tools=[file_read_tool],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

Create the tasks.

In [11]:
research_task = Task(
    description="""
    You are researching the podcast guest {guest_name} for the podcast {podcast_name}.
    The podcast description is as follows: {podcast_description}.
    
    First, read the following document to understand the podcast's purpose and structure: {podcast_introduction_filepath}.
    
    Then read more about the guest {guest_name} using the following documents (using the file_read_tool):
    {guest_linkedin_profile_filepath}
    {guest_linkedin_posts_filepath}
    
    Focus on:
    1. Their professional background and expertise in relation to the podcast's topic
    2. Recent projects, achievements, startups, products, and services related to the podcast's topic
    3. Their thought leadership and key ideas related to the podcast's topic
    4. Any unique perspectives or experiences they bring related to the podcast's topic
    
    VERY IMPORTANT: When you're compiling your findings, 
    MAKE SURE TO GIVE MORE IMPORTANCE TO RECENT FINDINGS. For example, if the guest has a recent project, 
    make sure to give more importance to that project. If you focus on the guest's project from 10 years ago that
    is discontinued, it's not useful.
    
    Compile your findings into a comprehensive research summary.""",
    expected_output="""A detailed research summary containing:
    1. Professional background and current role
    2. Key achievements and notable projects
    3. Areas of expertise and thought leadership
    4. Recent activities and public presence
    5. Unique perspectives or specialized knowledge
    
    The summary should combine information from provided documents and web research,
    ensuring all facts are verified and relevant to podcast discussion topics.""",
    agent=researcher,
)

outline_task = Task(
    description="""
    You are writing the podcast outline for guest {guest_name} for the podcast {podcast_name}.
    The podcast description is as follows: {podcast_description}.
    
    First, read the following document to understand the podcast's purpose and structure: {podcast_introduction_filepath}.
    
    Then using the research provided by the Senior Podcast Guest Researcher, create a one-pager podcast outline that:
    1. Starts with a note encouraging the guest to edit the document
    2. Includes a 2-3 sentence introduction about the guest
    3. Suggests 4-5 relevant segments tailored to the guest's expertise
    4. Provides brief descriptions and specific questions for each segment
    
    The outline should be conversational, engaging, and specifically tailored to the guest's 
    background and expertise. Ensure questions are clear and accessible.""",
    expected_output="""A structured podcast outline document containing:
    1. An opening note inviting guest feedback
    2. A concise guest introduction
    3. 4-5 clearly defined podcast segments
    4. Specific questions and talking points for each segment
    
    The outline should be personalized to the guest's expertise and maintain
    an engaging, conversational flow suitable for a podcast format.""",
    agent=outline_writer,
)

Create the crew.

In [12]:
podcast_crew = Crew(
    agents=[researcher, outline_writer],
    tasks=[research_task, outline_task],
    verbose=True,
    memory=True,
)

Execute the crew.

In [15]:
profile_id = "fergtech"
guest_name = "Fergus Mckenzie-Wilson"
profile = linkedin.get_profile(profile_id)
posts = linkedin.get_profile_posts(profile_id, post_count=100)

In [16]:
# Specifying the fields that are essential for posts (semantic content) -- otherwise it brings too much noisy metadata.
essential_post_fields = [
    'commentary.text.text',  # The actual post text written by the author
    'resharedUpdate.commentary.text.text',  # Text from reshared posts
    'actor.name.text',  # Author's name
    'resharedUpdate.actor.name.text',  # Name of original poster if reshared
    'actor.subDescription.text',  # Post timing (e.g., "4h • Edited")
]

os.makedirs(f"{collected_data_dir}/{profile_id}", exist_ok=True)
df_profile = pd.json_normalize(profile).T
df_profile.to_csv(f"{collected_data_dir}/{profile_id}/linkedin_profile.csv")
df_posts = pd.json_normalize(posts)[essential_post_fields].T
df_posts.to_csv(f"{collected_data_dir}/{profile_id}/linkedin_posts.csv")

In [18]:
inputs = {
    "podcast_name": "The AI Agents Podcast",
    "podcast_description": "A podcast about AI agents and how they are changing the world from scientific, commercial, ethical, and societal perspectives.",
    "guest_name": guest_name,
    "podcast_introduction_filepath": f"./{base_data_dir}/podcast_introduction.txt",
    "guest_linkedin_profile_filepath": f"./{collected_data_dir}/{profile_id}/linkedin_profile.csv",
    "guest_linkedin_posts_filepath": f"./{collected_data_dir}/{profile_id}/linkedin_posts.csv",
}

result = podcast_crew.kickoff(inputs=inputs)

# Agent: Senior Podcast Guest Researcher
## Task: 
    You are researching the podcast guest Fergus Mckenzie-Wilson for the podcast The AI Agents Podcast.
    The podcast description is as follows: A podcast about AI agents and how they are changing the world from scientific, commercial, ethical, and societal perspectives..
    
    First, read the following document to understand the podcast's purpose and structure: ./base_data/podcast_outline/podcast_introduction.txt.
    
    Then read more about the guest Fergus Mckenzie-Wilson using the following documents (using the file_read_tool):
    ./collected_data/podcast_outline/fergtech/linkedin_profile.csv
    ./collected_data/podcast_outline/fergtech/linkedin_posts.csv
    
    Focus on:
    1. Their professional background and expertise in relation to the podcast's topic
    2. Recent projects, achievements, startups, products, and services related to the podcast's topic
    3. Their thought leadership and key ideas related to the podc

/Users/uzaymacar/Library/Python/3.11/lib/python/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)


# Agent: Senior Podcast Outline Writer
## Task: 
    You are writing the podcast outline for guest Fergus Mckenzie-Wilson for the podcast The AI Agents Podcast.
    The podcast description is as follows: A podcast about AI agents and how they are changing the world from scientific, commercial, ethical, and societal perspectives..
    
    First, read the following document to understand the podcast's purpose and structure: ./base_data/podcast_outline/podcast_introduction.txt.
    
    Then using the research provided by the Senior Podcast Guest Researcher, create a one-pager podcast outline that:
    1. Starts with a note encouraging the guest to edit the document
    2. Includes a 2-3 sentence introduction about the guest
    3. Suggests 4-5 relevant segments tailored to the guest's expertise
    4. Provides brief descriptions and specific questions for each segment
    
    The outline should be conversational, engaging, and specifically tailored to the guest's 
    background and ex

/Users/uzaymacar/Library/Python/3.11/lib/python/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)


In [19]:
from IPython.display import Markdown
Markdown(result.raw)

THE AI AGENTS PODCAST - EPISODE OUTLINE
Guest: Fergus Mckenzie-Wilson

NOTE TO GUEST:
Dear Fergus,
We've crafted this outline based on your impressive background in AI and disinformation detection. Please feel free to modify any aspects to better align with your expertise and preferences. We want to ensure we capture the most valuable insights for our audience while maintaining a natural conversation flow.

GUEST INTRODUCTION:
Fergus Mckenzie-Wilson is the CTO of Valent, where he leads the development of cutting-edge AI-powered disinformation detection tools. With a background in Aeronautical Engineering and experience founding Hydra, a hybrid jet-electric drone technology company, Fergus brings a unique perspective on the evolution of AI systems. His current work focuses on pioneering multi-agent AI architectures for detecting and analyzing online manipulation, making him a leading voice in the practical application of AI agents for solving complex real-world problems.

SEGMENT 1: Journey into Multi-Agent AI Systems
Description: Exploring the evolution from hardware to AI-native solutions
Questions:
- How did your journey from aerospace engineering to AI development unfold?
- What inspired your transition from drone technology to AI-powered disinformation detection?
- How has your engineering background influenced your approach to AI architecture?

SEGMENT 2: Multi-Agent AI Architecture Deep Dive
Description: Understanding the technical foundations of modern AI agent systems
Questions:
- Could you explain how multi-agent AI systems differ from traditional AI approaches?
- What are the key components that make multi-agent systems effective?
- How do you manage the complexity of multiple AI agents working together?
- What breakthroughs have you seen in multi-agent cooperation?

SEGMENT 3: Battling Disinformation with AI Agents
Description: Exploring the practical applications of AI in detecting manipulation
Questions:
- How does Ariadne leverage AI agents to detect and analyze disinformation?
- What unique challenges exist in detecting financial market manipulation?
- Can you share examples of how AI agents have successfully identified manipulation patterns?
- What role do different types of AI agents play in the detection process?

SEGMENT 4: Commercial Applications and Scalability
Description: Discussing the business implementation of AI agent systems
Questions:
- How did you approach transforming Valent from a consultancy to an AI-Native SaaS company?
- What challenges arise when scaling AI agent systems for commercial use?
- How do you balance system complexity with practical business needs?
- What advice would you give to companies looking to implement multi-agent AI solutions?

SEGMENT 5: Future of AI Agents and Societal Impact
Description: Looking ahead at the evolution and implications of AI agent technology
Questions:
- Where do you see multi-agent AI systems heading in the next few years?
- What potential societal impacts should we be considering?
- How can we ensure responsible development of AI agent systems?
- What emerging applications of multi-agent AI excite you most?

Technical Notes:
- Feel free to share specific examples or case studies that illustrate your points
- We can include technical diagrams in the show notes if helpful
- Please let us know if you'd like to demonstrate any specific aspects of your work

Promotion and Social Media:
Please share your preferred social media handles and any links you'd like us to include in the show notes:
- LinkedIn: [Please provide]
- Company website: [Valent URL]
- Other relevant links you'd like to share

Thank you for joining The AI Agents Podcast! We look forward to an insightful discussion about multi-agent AI systems and their impact on detecting and countering disinformation.